Instalando o Kaggle

In [1]:
!pip install kaggle

Realizando o upload do token de segurança de comunicação com o kaggle

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 72 bytes


In [3]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        151           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        286           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        848           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       5134           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      40129           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      10535           False  
connectx

Fazendo o download do dataset do kaggle

In [4]:
!kaggle datasets download -d saraivaufc/automatic-weather-stations-brazil

automatic-weather-stations-brazil.zip: Skipping, found more recently modified local copy (use --force to force download)


Descompactando o dataset baixado

In [5]:
!unzip automatic-weather-stations-brazil.zip

Archive:  automatic-weather-stations-brazil.zip
replace automatic_stations_codes_2000_2021.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: automatic_stations_codes_2000_2021.csv  
replace automatic_weather_stations_inmet_brazil_2000_2021.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: automatic_weather_stations_inmet_brazil_2000_2021.csv  y
y



Configurando o Spark

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz
!tar xf spark-3.0.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

In [9]:
# Criando uma sessão no Spark
spark = SparkSession.builder.getOrCreate()

from datetime import datetime, date 
import pandas as pd
from pyspark.sql import Row

Carregando dataset no spark

In [15]:

# Realizando carga de arquivo texto bruto
#linhas = spark.sparkContext.textFile('automatic_weather_stations_inmet_brazil_2000_2021.csv')
#print(linhas.take(200))


# Realizando carga direta em dataset

df = spark.read.format("csv").option("header", "true").load("automatic_weather_stations_inmet_brazil_2000_2021.csv")

df.show(20)

+------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+---------------------------+--------------------------+-------------------------+
|ESTACAO;DATA (YYYY-MM-DD);HORA (UTC);PRECIPITACAO TOTAL HORARIO (mm);PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO| HORARIA (mB);PRESSAO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB);PRESSAO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB);RADIACAO GLOBAL (W/m2);TEMPERATURA DO AR - BULBO SECO| HORARIA (C);TE